In [1]:
import os
os.chdir("../src/SynBPS/")

# Make an experimental design table
Here the settings for the experiments can be modified in the dictionary called run_settings. Refer to the paper for more details on of each of the parameters.

In [2]:
run_settings = {# level of entropy: min, medium and/or max
                "process_entropy":["min_entropy"], #,"med_entropy","max_entropy"
                
                # number of traces/cases in the event-log
                "number_of_traces":[1000],

                # number of activity types
                "statespace_size":[5], 

                # first or higher-order markov chain to represent the transitions
                "process_type":["memoryless"], 
                
                # order of HOMC - only specify this when using process with memory
                "process_memory":[2],
                
                # number of transitions - only used for medium entropy (should be higher than 2 and < statespace size)
                "med_ent_n_transitions":[5],
                                
                # lambda parameter of inter-arrival times
                "inter_arrival_time":[1.5],
                
                # lambda parameter of process noise
                "process_stability_scale":[0.1],
                
                # probability of agent being available
                "resource_availability_p":[0.5],

                # number of agents in the process
                "resource_availability_n":[3],

                # waiting time in full days, when no agent is available
                "resource_availability_m":[0.041],
                
                # variation between activity durations
                "activity_duration_lambda_range":[1],
                
                # business hours definition: when can cases be processed? ()
                "Deterministic_offset_W":["weekdays"],

                # time-unit for a full week: days = 7, hrs = 24*7, etc.
                "Deterministic_offset_u":[7],
                
                # training data format (See Verenich et al., 2019): 
                # True - use first event to predict total cycle-time. 
                # False - use Prefix-log format / each event to predict remaining cycle time.
                "first_state_model":[False],

                # offset for the timestamps used (years after 1970)
                "datetime_offset":[45],
                
                # number of repetitions of the experiments: duplicates the experiment table (2 times here)
                "num_replications":list(range(0, 1))
               }


# import the make_design_table function to generate a full factorial experimental design table
from SynBPS.simulation.DoE import make_design_table
df = make_design_table(run_settings, file="data/design_table.csv")

# inspect the resulting design table
df

/Users/mikeriess/anaconda3/envs/SynBPS/lib/python3.9/site-packages/SynBPS/simulation/DoE.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'min_entropy' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/mikeriess/anaconda3/envs/SynBPS/lib/python3.9/site-packages/SynBPS/simulation/DoE.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'memoryless' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/mikeriess/anaconda3/envs/SynBPS/lib/python3.9/site-packages/SynBPS/simulation/DoE.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.1' has dtype incompatible with float32, please explicitly cast to a com

saved to data/design_table.csv


OSError: Cannot save file into a non-existent directory: 'data'

# Load design table 
If you chose to separate the generating of a design table with the experiment pipeline below, you can also load the generated design table. 

Its default location is in the results folder.

In [ ]:
df = pd.read_csv("../../tests/data/design_table.csv")

# Perform the experiments
Here you want to bring your own code into the end of the simulation loop. 

In [ ]:
# Placeholder for the results
results = []

# Iterate over each run in the design table df
for run in df.index:
    
    """
    Retrieving settings for experiment i
    """
    curr_settings = df.loc[run]
    curr_settings["run"] = run
    
    """
    If experiment is not previously performed
    """
    if curr_settings.Done == 0:
        print("Run:",run)
        start_time = time.time()

        # generate the log
        from simulation.simulation_pipeline import generate_eventlog

        log = generate_eventlog(curr_settings=curr_settings, output_dir="../../tests/data/")
        
        #log.to_csv("results/"+str(run)+"log.csv",index=False)

        # store metrics from simulated log
        curr_settings["simuation_time_sec"] = time.time() - start_time
        curr_settings["num_traces"] = len(log.caseid.unique())
        curr_settings["num_events"] = len(log)

        variants = []
        tracelengths = []
        for traceid in log.caseid.unique():
            trace = log.loc[log.caseid == traceid]
            #tracelen
            tracelen = len(trace)
            tracelengths.append(tracelen)
            #variant
            sequence = ""
            sequence = sequence.join(trace.activity.tolist())
            variants.append(sequence)

        # log simulated log characteristics
        n_variants = len(set(variants))       
        curr_settings["num_variants"] = n_variants
        curr_settings["avg_tracelen"] = np.mean(tracelengths)
        curr_settings["min_tracelen"] = np.min(tracelengths)
        curr_settings["max_tracelen"] = np.max(tracelengths)

        """
        Run the machine learning pipeline from here
        """

        if curr_settings["model_pipeline"] == True:
            """
            Prepare data for modelling (memory here refers to RAM)
            """
            from dataprep.memory_helperfunctions import prepare_data_f_memory
            input_data = prepare_data_f_memory(log, verbose=False)

            # if first-state model is desired, use the following function after generating the input_data object
            #from dataprep.firststate_helperfunctions import fs_prepare_dataset_from_memory
            #input_data = fs_prepare_dataset_from_memory(input_data, sample=1.0, transform="log", first_state=True, verbose=True)

            """
            Train a model
            """
            # X: 
            input_data["x_train"]
            input_data["x_test"]

            # Y:
            input_data["y_train"]
            input_data["y_test"]
            
            
            
            ### <<<<<< Insert training functions here >>>>>>>
            

            """
            Evaluate the model
            """

            ### <<<<<< Insert evaluation functions here >>>>>>>
            
            """
            Store the results
            """
            
            ### Store metrics to the in curr_settings dictionary which becomes the result table
            ### Prefixing column names is ideal for later analysis

            curr_settings["RESULT_num_events"] = len(log)
        
        #Mark as done in the design table
        df.loc[run,"Done"] = 1
        df.to_csv("../../tests/data/design_table.csv",index=False)  
        
        #Store the settings of run i
        results.append(curr_settings)
        
                
#store results
experiments = pd.DataFrame(results)
experiments.to_csv("../../tests/data/experiments.csv",index=False)  